# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import requests

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset used is the Iris dataset, containing 150 samples of iris flowers. Each sample is described by four features: sepal length, sepal width, petal length, and petal width. The target variable represents the species of the iris flower (Iris-setosa, Iris-versicolor, and Iris-virginica). Source: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris

The task is to use Azure AutoML to classify iris flower species based on their features. This involves:
- Loading and preparing the dataset
- Registering the dataset in AzureML workspace.
- Creating and running an AutoML experiment using the dataset
- Save the best model

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp_iris_automl'

experiment=Experiment(ws, experiment_name)

In [5]:
iris = load_iris()

# Combine features and target into a single DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Register the dataset
datastore = ws.get_default_datastore()
iris_dataset = Dataset.Tabular.register_pandas_dataframe(
    name='iris_dataset',
    dataframe=iris_df,
    target=(datastore, 'iris_dataset')
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to iris_dataset/1c1b4f59-56c8-4d0a-8ada-98f88cfc5cb8/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "iteration_timeout_minutes": 5,  # Each model iteration will run for a maximum of 5 minutes.
    "experiment_timeout_hours": 0.25,  # The entire experiment will run for a maximum of 15 minutes.
    "n_cross_validations": 5,  # Use 5-fold cross-validation to ensure model robustness.
    "primary_metric": 'accuracy',  # Optimize the model for accuracy.
    "enable_onnx_compatible_models": True  # Ensure models are ONNX-compatible for deployment.
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',  # The task is classification based on the Iris dataset.
    training_data=iris_dataset,  # The dataset to be used for training.
    label_column_name='target',  # The column name of the target variable.
    compute_target='notebook260384',  # The compute target where the experiment will run.
    path='.',  # The directory to save outputs.
    **automl_settings  # Include the automl_settings.
)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
exp_iris_automl,AutoML_92037499-0e8d-40a4-83e5-73ecfc90c7f0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run, fitted_model = remote_run.get_output()
print("Best Model: ", best_run)
print("Best Model Details: ", fitted_model)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Best Model:  Run(Experiment: exp_iris_automl,
Id: AutoML_92037499-0e8d-40a4-83e5-73ecfc90c7f0_12,
Type: None,
Status: Completed)
Best Model Details:  Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 StandardScalerWrapper(copy=True, with_mean=True, with_std=False)),
                ('LogisticRegression',
                 LogisticRegression(C=16.768329368110066, class_weight=None,
                                    dual=False, fit_intercept=True,
                                    intercept_scaling=1, l1_ratio=None,
                                    max_iter=100, multi_class='multinomial',
                                    n_job

In [10]:
#TODO: Save the best model
model = best_run.register_model(model_name='automl_best_model', model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# Retrieve the environment used in the AutoML run
env = best_run.get_environment()

# Inference configuration
inference_config = InferenceConfig(entry_script="score_automl.py", environment=env)

# Deploy the model
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service_name = "service-automl"

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-08 17:07:40+00:00 Creating Container Registry if not exists.
2024-06-08 17:07:40+00:00 Registering the environment.
2024-06-08 17:07:40+00:00 Use the existing image.
2024-06-08 17:07:41+00:00 Generating deployment configuration.
2024-06-08 17:07:41+00:00 Submitting deployment to compute.
2024-06-08 17:07:44+00:00 Checking the status of deployment service-automl..
2024-06-08 17:11:08+00:00 Checking the status of inference endpoint service-automl.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [15]:
test_sample = json.dumps({'data': [iris.data[0].tolist()]})
headers = {'Content-Type': 'application/json'}

response = requests.post(service.scoring_uri, data=test_sample, headers=headers)
print(response.text)

"{\"result\": [0]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [16]:
print(service.get_logs())

service.delete()

2024-06-08T17:10:04,437409174+00:00 - rsyslog/run 
2024-06-08T17:10:04,445003174+00:00 - gunicorn/run 
2024-06-08T17:10:04,453041674+00:00 | gunicorn/run | 
2024-06-08T17:10:04,455838174+00:00 | gunicorn/run | ###############################################
2024-06-08T17:10:04,460813574+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-06-08T17:10:04,462405574+00:00 - nginx/run 
2024-06-08T17:10:04,463025774+00:00 | gunicorn/run | ###############################################
2024-06-08T17:10:04,464695274+00:00 | gunicorn/run | 
2024-06-08T17:10:04,472060974+00:00 | gunicorn/run | 
2024-06-08T17:10:04,482951774+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-06-08T17:10:04,488905574+00:00 | gunicorn/run | 
2024-06-08T17:10:04,494282274+00:00 | gunicorn/run | 
2024-06-08T17:10:04,500357474+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

Running
2024-06-08 17:23:20+00:00 Check and wait for operation (4f3c971a-5883-412c-89f8-fd33c6f258e6) to finish.
2024-06-08 17:23:20+00:00 Deleting service.
2024-06-08 17:23:24+00:00 Deleting service entity.
Succeeded


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
